In [1]:
import calendar
from Core.Preprocess import *
from Core.CryptoCompareAPI import *
from Core.config_reader import config_reader
from Core.reduce_dim_with_correlation import correlation
from scipy import stats

## Download Data

In [2]:
reader = config_reader('/Users/admin/PycharmProjects/Market_Prediction_Using_News_and_Indicators/config.yml')

crypto_currency_list = reader.read_config("CRYPTO_CURRENCIES_LIST")
exchange = reader.read_config("EXCHANGE")
currency = reader.read_config("CURRENCY")

In [3]:
df_crypto_list = []

In [4]:
 for crypto in crypto_currency_list:
    raw_data = get_historical_price_hour_from(
        coin=crypto,
        currency=currency,
        exchange=exchange,
        toTs=int(calendar.timegm(datetime.datetime(2023, 7, 1, 0, 0, 0).timetuple())),
        fromTs=int(calendar.timegm(datetime.datetime(2023, 4, 1, 0, 0, 0).timetuple()))
    )
    data = Preprocess.get_from_crypto_compare_hourly_price_data_to_pandas(raw_data)
    df_crypto_list.append(data)

## Preprocessing

In [5]:
df_crypto_list[0]

,close,high,low,open,volumefrom,volumeto,conversionType,conversionSymbol,datetimes
time,,,,,,,,,
2023-04-01 00:00:00,28438.27,28547.20,28408.57,28465.36,1524.82,4.341578e+07,force_direct,,2023-04-01
2023-04-01 01:00:00,28610.63,28625.61,28410.35,28438.27,1458.59,4.157274e+07,force_direct,,2023-04-01
2023-04-01 02:00:00,28583.36,28819.71,28467.31,28610.63,3522.34,1.009208e+08,force_direct,,2023-04-01
2023-04-01 03:00:00,28541.10,28603.78,28503.29,28583.36,1088.36,3.107279e+07,force_direct,,2023-04-01
2023-04-01 04:00:00,28580.00,28625.10,28526.59,28541.10,1117.54,3.193753e+07,force_direct,,2023-04-01
...,...,...,...,...,...,...,...,...,...
2023-06-11 11:00:00,25796.06,25842.44,25757.13,25774.08,1164.36,3.004146e+07,force_direct,,2023-06-11
2023-06-11 12:00:00,25750.00,25798.72,25747.61,25796.06,809.95,2.087544e+07,force_direct,,2023-06-11
2023-06-11 13:00:00,25703.33,25776.13,25672.68,25750.00,1389.19,3.576042e+07,force_direct,,2023-06-11


In [6]:
for i in range(len(df_crypto_list)):
    Preprocess.rename_columns(df_crypto_list[i])

In [7]:
df_crypto_list[0]

,Close,High,Low,Open,Volume
time,,,,,
2023-04-01 00:00:00,28438.27,28547.20,28408.57,28465.36,4.341578e+07
2023-04-01 01:00:00,28610.63,28625.61,28410.35,28438.27,4.157274e+07
2023-04-01 02:00:00,28583.36,28819.71,28467.31,28610.63,1.009208e+08
2023-04-01 03:00:00,28541.10,28603.78,28503.29,28583.36,3.107279e+07
2023-04-01 04:00:00,28580.00,28625.10,28526.59,28541.10,3.193753e+07
...,...,...,...,...,...
2023-06-11 11:00:00,25796.06,25842.44,25757.13,25774.08,3.004146e+07
2023-06-11 12:00:00,25750.00,25798.72,25747.61,25796.06,2.087544e+07
2023-06-11 13:00:00,25703.33,25776.13,25672.68,25750.00,3.576042e+07


In [8]:
def sign(x):
    return round((x + 1e-10) / (abs(x) + 1e-10), 1)

target_list = []

for df in df_crypto_list:
    target_list.append(sign(df['Close'].shift(-24) - df['Close'])[90:-24])

In [9]:
target_list[0]

time
2023-04-04 18:00:00   -1.0
2023-04-04 19:00:00    1.0
2023-04-04 20:00:00   -1.0
2023-04-04 21:00:00   -1.0
2023-04-04 22:00:00   -1.0
                      ... 
2023-06-10 11:00:00    1.0
2023-06-10 12:00:00    1.0
2023-06-10 13:00:00    1.0
2023-06-10 14:00:00    1.0
2023-06-10 15:00:00    1.0
Name: Close, Length: 1606, dtype: float64

## Compute indicators

In [10]:
new_df_crypto_list = list(map(Preprocess.compute_all_indicators, df_crypto_list))

In [11]:
for i in range(len(new_df_crypto_list)):
    new_df_crypto_list[i].drop(columns=["Close", "High", "Low", "Open", "Volume"], inplace=True)
    new_df_crypto_list[i] = new_df_crypto_list[i].iloc[90:-24]

In [12]:
for i in range(len(df_crypto_list)):
    df_crypto_list[i].drop(columns=["UpperBB", "LowerBB"], inplace = True)
    df_crypto_list[i] = df_crypto_list[i].iloc[90:-24]

In [13]:
df_crypto_list[0]

,Close,High,Low,Open,Volume
time,,,,,
2023-04-04 18:00:00,28114.42,28260.00,28083.33,28226.99,48084138.19
2023-04-04 19:00:00,28189.25,28199.00,28070.70,28114.42,40961476.75
2023-04-04 20:00:00,28250.04,28305.00,28174.44,28189.25,48298634.85
2023-04-04 21:00:00,28223.62,28254.11,28160.51,28250.04,29109712.30
2023-04-04 22:00:00,28187.26,28258.93,28182.22,28223.62,24092669.65
...,...,...,...,...,...
2023-06-10 11:00:00,25731.40,25739.29,25659.07,25698.31,30764553.12
2023-06-10 12:00:00,25616.18,25731.41,25608.58,25731.40,39339506.49
2023-06-10 13:00:00,25657.49,25679.06,25572.94,25616.18,31253570.80


In [14]:
new_df_crypto_list[0]

,UpperBB,LowerBB,DEMA,EMA,HT_TRENDLINE,KAMA,MA,MIDPOINT,MIDPRICE,SAR,...,STOCHRSI FASTD,TRIX,ULTOSC,WILLR,AD,ADOSC,OBV,TRANGE,ATR,NATR
time,,,,,,,,,,,,,,,,,,,,,
2023-04-04 18:00:00,28377.046239,27690.554761,28069.986877,28066.313562,27987.808174,28077.185305,28007.634667,28089.765,28129.150,28819.71,...,53.944825,-0.019340,43.650444,-52.335945,-9.738522e+07,-1.968933e+07,1.373518e+08,176.67,209.146034,0.002646
2023-04-04 19:00:00,28384.626967,27721.899033,28085.100953,28074.244945,27999.931609,28077.885185,28005.317000,28155.410,28152.305,28819.71,...,69.007400,-0.017924,47.364655,-43.676725,-6.264937e+07,-6.188136e+06,1.783133e+08,128.30,203.371318,0.002559
2023-04-04 20:00:00,28389.822077,27765.649923,28106.352081,28085.586562,28017.770739,28080.437568,28013.817000,28155.410,28187.820,28819.71,...,68.969806,-0.016442,52.966516,-37.877017,-5.501393e+07,2.327096e+06,2.266119e+08,130.56,198.170509,0.002483
2023-04-04 21:00:00,28390.898892,27805.319108,28122.248596,28094.491945,28037.541348,28081.974286,28019.555000,28159.780,28187.820,28819.71,...,81.988357,-0.014945,57.152220,-43.024706,-4.486907e+07,8.826956e+06,1.975022e+08,93.60,190.701187,0.002394
2023-04-04 22:00:00,28386.726415,27842.482585,28132.041784,28100.476981,28057.095087,28083.610004,28028.102333,28159.780,28196.015,28819.71,...,71.045138,-0.013484,55.565123,-51.762101,-6.579586e+07,4.025012e+06,1.734095e+08,76.71,182.558959,0.002298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-10 11:00:00,26923.863031,25322.940969,25874.467675,26139.895934,26185.485799,25963.846462,26278.727000,26005.375,25995.140,31000.00,...,98.777806,-0.021071,45.451903,-74.477939,2.795244e+08,-4.207018e+07,-2.558505e+09,80.22,154.057619,0.002327
2023-06-10 12:00:00,26892.346934,25267.791066,25826.195708,26106.107810,26164.120107,25925.480816,26249.502000,25997.935,25995.140,31000.00,...,81.130917,-0.024418,47.872389,-85.171607,2.450531e+08,-3.914038e+07,-2.597845e+09,122.83,151.827075,0.002314
2023-06-10 13:00:00,26846.568847,25229.712153,25788.235680,26077.164725,26134.199081,25898.963796,26221.637333,25989.025,25995.140,31000.00,...,79.568733,-0.027703,51.234049,-81.337590,2.636014e+08,-2.848187e+07,-2.566591e+09,106.12,148.562284,0.002257


## Reduce dim with Correlation

In [15]:
red_df_list = []

for i in range(len(new_df_crypto_list)):
    red_df_list.append(correlation.compute_correlation(new_df_crypto_list[i], "spearman", False))
    red_df_list[i] = new_df_crypto_list[i].drop(columns=correlation.reducing_dimension(red_df_list[i], 0.75))

In [16]:
red_df_list[0]

,UpperBB,SAR,ADX,APO,Aroon Up,Aroon Down,BOP,CCI,DX,STOCH SLOWK,TRIX,ULTOSC,AD,TRANGE,ATR
time,,,,,,,,,,,,,,,
2023-04-04 18:00:00,28377.046239,28819.71,12.848769,150.677244,72.0,16.0,-0.637177,54.364140,4.055920,49.611762,-0.019340,43.650444,-9.738522e+07,176.67,209.146034
2023-04-04 19:00:00,28384.626967,28819.71,12.133606,156.874231,68.0,12.0,0.583242,49.407982,2.836497,64.216470,-0.017924,47.364655,-6.264937e+07,128.30,203.371318
2023-04-04 20:00:00,28389.822077,28819.71,12.134229,165.316795,64.0,8.0,0.465610,86.627515,12.142316,71.431973,-0.016442,52.966516,-5.501393e+07,130.56,198.170509
2023-04-04 21:00:00,28390.898892,28819.71,12.027685,161.906154,60.0,4.0,-0.282265,64.586351,10.642618,75.387230,-0.014945,57.152220,-4.486907e+07,93.60,190.701187
2023-04-04 22:00:00,28386.726415,28819.71,11.960401,135.959423,56.0,0.0,-0.473993,57.689269,11.085703,66.185466,-0.013484,55.565123,-6.579586e+07,76.71,182.558959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-10 11:00:00,26923.863031,31000.00,33.979583,-340.152372,12.0,76.0,0.412491,-76.942510,48.147735,76.834053,-0.021071,45.451903,2.795244e+08,80.22,154.057619
2023-06-10 12:00:00,26892.346934,31000.00,35.191585,-371.269103,8.0,72.0,-0.938044,-77.638228,50.947606,67.996205,-0.024418,47.872389,2.450531e+08,122.83,151.827075
2023-06-10 13:00:00,26846.568847,31000.00,36.455164,-396.246795,4.0,68.0,0.389276,-72.295459,52.881697,57.518287,-0.027703,51.234049,2.636014e+08,106.12,148.562284


## Normalization with z-score

In [17]:
normalization_df_list = [stats.zscore(new_df_crypto_list[i]) for i in range(len(new_df_crypto_list))]
normalization_df_list[0]

,UpperBB,LowerBB,DEMA,EMA,HT_TRENDLINE,KAMA,MA,MIDPOINT,MIDPRICE,SAR,...,STOCHRSI FASTD,TRIX,ULTOSC,WILLR,AD,ADOSC,OBV,TRANGE,ATR,NATR
time,,,,,,,,,,,,,,,,,,,,,
2023-04-04 18:00:00,0.059091,0.093718,0.113341,0.103322,0.036689,0.094135,0.051981,0.129042,0.168155,-0.926610,...,0.129885,-0.352792,-0.942380,-0.082621,-2.015408,-0.617139,0.782545,0.103678,0.787675,0.792643
2023-04-04 19:00:00,0.065134,0.120079,0.125568,0.110108,0.046952,0.094709,0.050018,0.183726,0.187406,-0.926610,...,0.615420,-0.320545,-0.420852,0.272536,-1.927296,-0.209577,0.821738,-0.251165,0.688839,0.669968
2023-04-04 20:00:00,0.069275,0.156874,0.142758,0.119813,0.062055,0.096805,0.057216,0.183726,0.216934,-0.926610,...,0.614208,-0.286798,0.365728,0.510410,-1.907928,0.047474,0.867951,-0.234586,0.599825,0.562194
2023-04-04 21:00:00,0.070134,0.190237,0.155617,0.127433,0.078793,0.098066,0.062075,0.187366,0.216934,-0.926610,...,1.033855,-0.252697,0.953460,0.299278,-1.882194,0.243686,0.840098,-0.505725,0.471984,0.436100
2023-04-04 22:00:00,0.066808,0.221491,0.163540,0.132554,0.095347,0.099409,0.069314,0.187366,0.223748,-0.926610,...,0.681105,-0.219413,0.730609,-0.059085,-1.935278,0.098729,0.817046,-0.629631,0.332627,0.299845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-10 11:00:00,-1.099264,-1.897473,-1.662688,-1.545022,-1.489154,-1.640871,-1.412118,-1.607307,-1.606091,0.563495,...,1.575054,-0.392214,-0.689430,-0.990772,-1.059329,-1.292754,-1.796910,-0.603881,-0.155185,0.340968
2023-06-10 12:00:00,-1.124386,-1.943855,-1.701737,-1.573933,-1.507242,-1.672368,-1.436866,-1.613504,-1.606091,0.563495,...,1.006215,-0.468441,-0.349559,-1.429371,-1.146770,-1.204311,-1.834551,-0.291293,-0.193362,0.322554
2023-06-10 13:00:00,-1.160876,-1.975879,-1.732444,-1.598699,-1.532573,-1.694138,-1.460463,-1.620927,-1.606091,0.563495,...,0.955859,-0.543260,0.122465,-1.272119,-1.099720,-0.882561,-1.804647,-0.413878,-0.249240,0.241849


## Save Data

In [18]:
PATH = "/Users/admin/PycharmProjects/Market_Prediction_Using_News_and_Indicators/Crypto_Currencies_Data/"

In [19]:
for i in range(len(crypto_currency_list)):
    new_df_crypto_list[i].to_csv(PATH + crypto_currency_list[i] + "_indicators.csv")
    df_crypto_list[i].to_csv(PATH + crypto_currency_list[i] + ".csv")
    red_df_list[i].to_csv(PATH + crypto_currency_list[i] + "_reduce.csv")
    normalization_df_list[i].to_csv(PATH + crypto_currency_list[i] + "_normalization.csv")
    target_list[i].to_csv(PATH + crypto_currency_list[i] + "_target.csv")